In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import plotly.offline as py
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import cufflinks as cf
cf.set_config_file(offline=True, sharing=False, theme='solar')

# Creating Synthetic Data

For this lecture we use a simple synthetic dataset to simplify the presentation of ideas.

In [ ]:
n = 100
np.random.seed(42)
noise = 0.7
w_true = np.array([1., 3.])
quad = -4

x = np.sort(np.random.rand(n)*2 - 1.)
y = w_true[0] + w_true[1] * x + quad*(x**2) + noise * np.random.randn(n)
x[0] = -1.5
y[0] = 10

In [ ]:
raw_data_plot = go.Scatter(x=x, y=y, name="Raw Data", mode='markers')
fig = go.Figure([raw_data_plot])
fig.update_layout(height=600)

# Defining The Model and Loss Functions

Starting with a simple linear model:

In [ ]:
def model(w, x):
    return w[0] + w[1] * x

From the previous lecture we showed how to analytically derive the minimizer for the squared loss:

In [ ]:
def standard_units(x):
    return (x - np.mean(x)) / np.std(x)

def correlation(x, y):
    return np.mean (standard_units(x) * standard_units(y))

def slope(x, y):
    return correlation(x, y) * np.std(y) / np.std(x)

def intercept(x, y):
    return np.mean(y) - slope(x, y)*np.mean(x)

Computing the weights based on the functions from prior lecture.

In [ ]:
w_mse = np.array([intercept(x,y), slope(x,y)])
w_mse

In [ ]:
y_hat = model(w_mse, x)
analytic_mse_line = go.Scatter(x=x,y=y_hat, name="Analytic MSE")
fig = go.Figure([raw_data_plot, analytic_mse_line])
fig.update_layout(height=700)
fig

# Visualizing the Residuals

The following code is for Plotly only to generate residual plots.

In [ ]:
def residual_lines(x, y, yhat):
    return [ 
        go.Scatter(x=[x,x], y=[y,yhat],
               mode='lines', showlegend=False, 
               line=dict(color='black', width = 0.5))
        for (x, y, yhat) in zip(x, y, y_hat)
    ]

In [ ]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
for t in residual_lines(x,y,y_hat) + [raw_data_plot, analytic_mse_line]:
    fig.add_trace(t, row=1,col=1)
fig.add_trace(go.Scatter(x=x, y =   y -y_hat , mode='markers', name='Residuals'), row=2, col=1)
fig.add_trace(go.Scatter(x=[x.min(), x.max()], y = [0,0], showlegend=False), row=2, col=1)
fig.update_yaxes(title_text="Y", row=1, col=1)
fig.update_yaxes(title_text="Residual", row=2, col=1)
fig.update_layout(height=700)

# Visualizing the Loss Surface

In [ ]:
def mse_loss(yhat, y):
    return ((yhat - y)**2).mean()

Exhaustively try a large number of possible parameter values.

In [ ]:
# Compute Range of intercpet values
w0values = np.linspace(w_mse[0]-1, w_mse[0]+1, 50) 
# Compute Range of Slope values
w1values = np.linspace(w_mse[1]-1, w_mse[1]+1, 50)
# Construct "outer product of all possible values"
(u,v) = np.meshgrid(w0values, w1values)
# Convert into a tall matrix with each row corresponding to a possible parameterization
ws = np.vstack((u.flatten(),v.flatten())).transpose()
# Compute the Loss for each parameterization
mse_loss_values = np.array([mse_loss(y, model(w, x)) for w in ws]).reshape(u.shape)

Make a really cool looking visualization of the loss surface.

In [ ]:
fig = make_subplots(rows=1, cols=2,
                    specs=[[{'type': 'contour'}, {'type': 'surface'}]])
# Make Contour Plot and Point
fig.add_trace(go.Contour(x=w0values, y=w1values, z=mse_loss_values , colorbar=dict(x=-.1)), row=1, col=1)
fig.add_trace(go.Scatter(x=[w_mse[0]], y=[w_mse[1]]), row=1, col=1)
# Make Surface Plot and Point
fig.add_trace(go.Surface(x=w0values, y=w1values, z=mse_loss_values, opacity=0.9), row=1, col=2)
fig.add_trace(go.Scatter3d(x=[w_mse[0]], y=[w_mse[1]], z=[mse_loss(y, model(w_mse,x))]), row=1, col=2)
# Cleanup Legend
fig.update_layout(scene=dict(xaxis=dict(title='Slope'), yaxis=dict(title='Intercept'), zaxis=dict(title="MSE Loss")))
fig.update_xaxes(title_text="Intercept", row=1, col=1)
fig.update_yaxes(title_text="Slope", row=1, col=1)
fig.update_layout(height=700)


# Examining the $L^1$ Loss

We just solved the model for the $L^2$ loss.  We now examine the $L^1$ loss.  We first begin by visualizing the loss surface.

In [ ]:
def abs_loss(yhat, y):
    return (np.abs(yhat - y)).mean()

In [ ]:
# Compute Range of intercpet values
w0values = np.linspace(w_mse[0]-1, w_mse[0]+1, 50) 
# Compute Range of Slope values
w1values = np.linspace(w_mse[1]-1, w_mse[1]+1, 50)
# Construct "outer product of all possible values"
(u,v) = np.meshgrid(w0values, w1values)
# Convert into a tall matrix with each row corresponding to a possible parameterization
ws = np.vstack((u.flatten(),v.flatten())).transpose()
# Compute the Loss for each parameterization
abs_loss_values = np.array([abs_loss(y, model(w, x)) for w in ws]).reshape(u.shape)

In [ ]:
fig = make_subplots(rows=1, cols=2,
                    specs=[[{'type': 'contour'}, {'type': 'surface'}]])
# Make Contour Plot and Point
fig.add_trace(go.Contour(x=w0values, y=w1values, z=abs_loss_values, colorbar=dict(x=-.1)), row=1, col=1)
fig.add_trace(go.Scatter(x=[w_mse[0]], y=[w_mse[1]]), row=1, col=1)
# Make Surface Plot and Point
fig.add_trace(go.Surface(x=w0values, y=w1values, z=abs_loss_values, opacity=0.9), row=1, col=2)
fig.add_trace(go.Scatter3d(x=[w_mse[0]], y=[w_mse[1]], z=[abs_loss(y, model(w_mse,x))]), row=1, col=2)
# Cleanup Legend
fig.update_layout(scene=dict(xaxis=dict(title='Slope'), yaxis=dict(title='Intercept'), zaxis=dict(title="MSE Loss")))
fig.update_xaxes(title_text="Intercept", row=1, col=1)
fig.update_yaxes(title_text="Slope", row=1, col=1)
fig.update_layout(height=700)

In [ ]:
fig = go.Figure([raw_data_plot, analytic_mse_line])
fig.update_layout(height=700)
fig

# Quick Introduction to Algorithmic Differentiation

In this lecture we are going to introduce PyTorch.  PyTorch is sort of like learning how to use Thor's hammer, it is way overkill for basically everything you will do and is probably the wrong solution to most problems you will encounter. However, it also really powerful and will give you the skills needed to take on very challenging problems. 

In [ ]:
import torch

Defining a variable $\theta$ with an initial value 1.0

In [ ]:
theta = torch.tensor([1.0], requires_grad=True, dtype=torch.float64)
theta

Suppose we compute the following value from our tensor `theta`

$$
z = \left(1 - log\left(1 + \exp(\theta) \right) \right)^2
$$

In [ ]:
z = (1 - torch.log(1 + torch.exp(theta)))**2
z

Notice that every derived value has an attached a gradient function that is used to compute the backwards path.

In [ ]:
z.grad_fn

In [ ]:
z.grad_fn.next_functions

In [ ]:
z.grad_fn.next_functions[0][0].next_functions

We can visualize these functions 

In [ ]:
# !pip install torchviz
# !brew install graphviz
# from torchviz import make_dot
# make_dot(z)

If you were unable to run the above cell here is what the output looks like:

<img src="torch_graph.png" alt="TorchGraph" style="height:400px;">


These backward functions tell Torch how to compute the gradient via the chain rule.  This is done by invoking backward on the computed value.

In [ ]:
z.backward()

In [ ]:
theta.grad

We can use `item` to extract a single value.

In [ ]:
theta.grad.item()

We can compare this witht he hand computed derivative:

\begin{align}
\frac{\partial z}{\partial\theta} &= \frac{\partial}{\partial\theta}\left(1 - \log\left(1 + \exp(\theta)\right)\right)^2 \\
 & = 2\left(1 - \log\left(1 + \exp(\theta)\right)\right)\frac{\partial}{\partial\theta} \left(1 - \log\left(1 + \exp(\theta)\right)\right)\\
 & = 2\left(1 - \log\left(1 + \exp(\theta)\right)\right) (-1) \frac{\partial}{\partial\theta} \log\left(1 + \exp(\theta)\right) \\
 & = 2\left(1 - \log\left(1 + \exp(\theta)\right)\right)   \frac{-1}{1 + \exp(\theta)}\frac{\partial}{\partial\theta}\left(1 + \exp(\theta)\right) \\
 & = 2\left(1 - \log\left(1 + \exp(\theta)\right)\right)   \frac{-1}{1 + \exp(\theta)}\exp(\theta) \\
  & = -2\left(1 - \log\left(1 + \exp(\theta)\right)\right)   \frac{\exp(\theta)}{1 + \exp(\theta)}
\end{align}

In [ ]:
def z_derivative(theta):
    return -2 * (1 - np.log(1 + np.exp(theta))) * np.exp(theta) / (1. + np.exp(theta))

In [ ]:
z_derivative(1.)

# Minimizing the Absolute Loss Using Gradient Descent

Here we will use pytorch to implement gradient descent.

In [ ]:
import torch

Converting numpy data to tensors.  I am storing these tensors in a dictionary data so I don't confuse them with my other data.

In [ ]:
data = dict(
    x = torch.from_numpy(x),
    y = torch.from_numpy(y))

## Defining a Model 
This is the most basic way to define a model.

In [ ]:
class SimpleLinearModel:
    def __init__(self):
        self.w = torch.zeros(2, 1, requires_grad=True)
    
    def predict(self, x):
        w = self.w
        return w[0] + w[1] * x

    def __call__(self, x):
        return self.predict(x)

In [ ]:
lin_model = SimpleLinearModel()
lin_model(data['x'])

## Computing the Loss

There are many built in loss functions but we will build our own to see how it all works.

In [ ]:
def abs_loss_torch(ypred, y):
    return torch.abs(ypred - y).mean()

In [ ]:
loss = abs_loss_torch(lin_model(data['x']), data['y'])
loss

In [ ]:
loss.backward()

In [ ]:
lin_model.w.grad

In [ ]:
lin_model.w.grad.zero_()

# Implementing Basic Gradient Descent

The following function implements a basic version of gradient descent. 

In [ ]:
def gradient_descent(model, loss_fn, x, y, lr=1., nsteps=100):
    values = [model.w.data.numpy().flatten()] # Track parameter values for later viz.
    for i in range(nsteps):
        loss = loss_fn(model(x), y)
        loss.backward()
        # We compute the update in a torch.no_grad context to prevent torch from 
        # trying to compute the gradient of the gradient calculation.
        with torch.no_grad():
            model.w -= lr * model.w.grad
            # We also need to clear the gradient buffer otherwise future calls will
            # accumulate the gradient. 
            model.w.grad.zero_()
            # print(i, loss.item())
            values.append(model.w.data.numpy().flatten())
    return np.array(values)

In [ ]:
lin_model = SimpleLinearModel()
values = gradient_descent(lin_model, abs_loss_torch, data['x'], data['y'], nsteps=50)
print(abs_loss_torch(data['y'], lin_model(data['x'])))
w_abs = lin_model.w.data.numpy().flatten()
print(w_abs)

In [ ]:
w0values = np.linspace(-1., 1, 50)
w1values = np.linspace(-.5, 5, 50)
(u,v) = np.meshgrid(w0values, w1values)
ws = np.vstack((u.flatten(),v.flatten())).transpose()
loss = np.array([abs_loss(y, model(w, x)).item() for w in ws]).reshape(u.shape)
fig = go.Figure([go.Contour(x=w0values, y=w1values, z=loss, colorbar=dict(x=-.1)),
          go.Scatter(x=[w_mse[0]], y=[w_mse[1]], name="MSE", mode="markers"),
          go.Scatter(x=[w_abs[0]], y=[w_abs[1]], name="Abs", mode="markers"),
          go.Scatter(x=values[:,0], y=values[:,1], name="Path", mode="markers+lines", 
                     line=go.scatter.Line(color='white'))])
fig.update_layout(height=600, xaxis_title="Intercept", yaxis_title="Slope")

Visualizing the current best fit line

In [ ]:
y_hat = model(w_abs, x)
gd_abs_line = go.Scatter(x=x,y=y_hat, name="GD ABS")
fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
for t in residual_lines(x,y,y_hat) + [raw_data_plot, analytic_mse_line, gd_abs_line]:
    fig.add_trace(t, row=1,col=1)
fig.add_trace(go.Scatter(x=x, y = y - y_hat, mode='markers', name='Residuals'), row=2, col=1)
fig.add_trace(go.Scatter(x=[x.min(), x.max()], y = [0,0], showlegend=False), row=2, col=1)
fig.update_yaxes(title_text="Y", row=1, col=1)
fig.update_yaxes(title_text="Residual", row=2, col=1)
fig.update_layout(height=600)

# Improving The Model

In the above we notice some curvature in the residual plot and decide to build a more complex model with an extra quadratic term.

In [ ]:
class QuadraticModel:
    def __init__(self):
        self.w = torch.zeros(3, 1, requires_grad=True)
    def predict(self, x):
        w = self.w
        return w[0] + w[1] * x + w[2] * (x * x)
    def __call__(self, x):
        return self.predict(x)

In [ ]:
quad_model = QuadraticModel()
values = gradient_descent(quad_model, abs_loss_torch, data['x'], data['y'], nsteps=1000)
print(abs_loss_torch(data['y'], quad_model(data['x'])).item())
w_quad_abs = quad_model.w.data.numpy().flatten()
print(w_quad_abs)

In [ ]:
with torch.no_grad():
    y_hat = quad_model(data['x']).numpy().flatten()

gd_quad_abs_line = go.Scatter(x=x, y=y_hat, name="GD Quad ABS")
fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
for t in residual_lines(x,y,y_hat) + [raw_data_plot, analytic_mse_line, gd_abs_line, gd_quad_abs_line]:
    fig.add_trace(t, row=1,col=1)
fig.add_trace(go.Scatter(x=x, y = y - y_hat , mode='markers', name='Residuals'), row=2, col=1)
fig.add_trace(go.Scatter(x=[x.min(), x.max()], y = [0,0], showlegend=False), row=2, col=1)
fig.update_yaxes(title_text="Y", row=1, col=1)
fig.update_yaxes(title_text="Residual", row=2, col=1)
fig.update_layout(height=600)

# Implementing Stochastic Gradient Descent

First we need a mechanism to sample the data.  Since this is central to SGD (and therefore PyTorch) there is a built in way to do this:

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
dataset = TensorDataset(data['x'], data['y'])
loader = DataLoader(dataset, batch_size=20, shuffle=True)

In [ ]:
[dict(x=x, y=y) for x, y in loader]

Basic SGD implementation

In [ ]:
def stochastic_gradient_descent(model, loss_fn, dataset, lr=1., nepochs=100, batch_size=10):
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    values = [model.w.data.numpy().flatten()] # Track parameter values for later viz.
    for i in range(nepochs):
        for (x, y) in loader:
            loss = loss_fn(model(x), y)
            loss.backward()
            # We compute the update in a torch.no_grad context to prevent torch from 
            # trying to compute the gradient of the gradient calculation.
            with torch.no_grad():
                model.w -= lr/(i+1) * model.w.grad
                # We also need to clear the gradient buffer otherwise future calls will
                # accumulate the gradient. 
                model.w.grad.zero_()
                # print(i, loss.item())
                values.append(model.w.data.numpy().flatten())
    return np.array(values)

In [ ]:
quad_model_sgd = QuadraticModel()
values = stochastic_gradient_descent(quad_model_sgd, abs_loss_torch, dataset, 
                                     lr=1.0, nepochs=20, batch_size=10)
print(abs_loss_torch(data['y'], quad_model_sgd(data['x'])).item())
w_quad_abs_sgd = quad_model_sgd.w.data.numpy().flatten()
print(w_quad_abs_sgd)

In [ ]:
fig = go.Figure([go.Scatter3d(x=values[:,0], y=values[:,1], z=values[:,2], 
                              marker=dict(color=np.linspace(0,1,values.shape[0]))
                             )])
fig.update_layout(height = 800)
py.iplot(fig)